**Facultad de Ciencias Exactas, Ingeniería y Agrimensura - UNR**

Tecnicatura Universitaria en Inteligencia Artificial

# Procesamiento del Lenguaje Natural - Trabajo Práctico N°: 1.

Integrantes:
- Pace, Bruno. Legajo: P-5295/7.
- Sancho Almenar, Mariano. Legajo: S-5778/9.

In [1]:
# ==========================
# Instalación de Paquetes Externos
# ==========================
!pip install sentiment_analysis_spanish
!pip install transformers
!pip install https://github.com/explosion/spacy-models/releases/download/es_core_news_lg-3.5.0/es_core_news_lg-3.5.0.tar.gz
!pip install gliner
!python -m spacy download en_core_web_sm

# ==========================
# Importación de Librerías
# ==========================
# Librerías básicas para manejo de datos
import pandas as pd
import warnings

# NLP y procesamiento de texto
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Modelos y herramientas de ML
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Análisis de sentimientos en español
from sentiment_analysis_spanish import sentiment_analysis

# Librerías de transformers
from gliner import GLiNER
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

# Configuración de warnings
warnings.filterwarnings('ignore')

# Descarga de recursos adicionales de NLTK
nltk.download('punkt')
nltk.download('stopwords')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.0/30.0 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.6/152.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.6/919.6 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.2 MB/s eta 0:00:00
  Created wheel for es_core_news_lg: filename=es_core_news_lg-3.5.0-py3-none-any.whl size=567975316 sha256=84c435ae60a4b05af6287f9439275402fcd0e76267b2f60a229067db3a76a39b
  Stored in directory: /root/.cache/pip/wheels/6c/1c/88/21043ad68f0560b1dda

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True


# Carga del dataset

In [61]:
df: pd.DataFrame = pd.read_csv('dataset.csv', index_col=0)

In [62]:
df.sample(5)

,Title,Description,Category,Type
1742,El secreto de sus ojos,A retired legal counselor writes a novel hopin...,"['Drama', 'Mystery', 'Romance']",Pelicula
2307,"The Mahabharata of Krishna-Dwaipayana Vyasa, V...","""The Mahabharata of Krishna-Dwaipayana Vyasa, ...","['Epic literature, Sanskrit']",Libro
871,Exit: The Game – The Secret Lab,"As the subjects of a medical study, the player...","['Deduction', 'Puzzle', 'Real-time']",Juego
1524,Trust,A teenage girl is targeted by an online sexual...,"['Crime', 'Drama', 'Thriller']",Pelicula
741,Churchill,The players in the game take on the roles of C...,"['Political', 'Wargame', 'World War II']",Juego



## Procesado de la columna 'Description'

In [63]:
df_filtered = df.copy()


### Remoción de stopwords con nltk

In [64]:
nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))

def remove_stopwords(text: str) -> str:
    """
    Remueve stopwords de un texto en ingles. Retorna el texto sin stopwords.
    """
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.casefold() not in stop_words]
    return " ".join(filtered_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [53]:
df_filtered['Lemmatized_Description'] = df_filtered['Description'].apply(remove_stopwords)


## Lematización del texto

Se utiliza spacy para la lematizacion del texto, el resultado será la descripción con las palabras lematizadas, sin puntuaciones o stopwords

In [65]:
lemmatizer = spacy.load("en_core_web_sm")

def text_lemmatizer(texto: str) -> str:
    """
    Función para lematizar un texto en inglés
    """
    lemmatizer_ = lemmatizer(texto)
    lemmas = [tok.lemma_.lower() for tok in lemmatizer_ if not tok.is_stop and not tok.is_punct]
    return ' '.join(lemmas)

In [56]:
df_filtered['Lemmatized_Description'] = df_filtered['Description'].apply(text_lemmatizer)

In [67]:
df_filtered.sample(8)

,Title,Description,Category,Type
2643,Plays of Sophocles: Oedipus the King; Oedipus ...,"""Plays of Sophocles: Oedipus the King; Oedipus...",['Tragedies'],Libro
1979,The Skin I Live In,"A brilliant plastic surgeon, haunted by past t...","['Drama', 'Thriller']",Pelicula
390,Furnace,Furnace is an engine-building Eurogame in whic...,"['Card Game', 'Economic', 'Industry / Manufact...",Juego
138,Forbidden Stars,The shifting Warp Storms that surround the lon...,"['Bluffing', 'Fighting', 'Miniatures', 'Scienc...",Juego
129,Age of Steam,Steam-belching iron horses roar across the wil...,"['Economic', 'Post-Napoleonic', 'Trains', 'Tra...",Juego
1032,X-Men: Apocalypse,After the re-emergence of the world's first mu...,"['Action', 'Adventure', 'Sci-Fi']",Pelicula
2050,Ulysses by James Joyce,"""Ulysses"" by James Joyce is a modernist novel ...",['City and town life -- Fiction'],Libro
2667,"The Vision and Creed of Piers Ploughman, Volum...","""The Vision and Creed of Piers Ploughman, Volu...",['Christian pilgrims and pilgrimages -- Poetry'],Libro



# Modelo

In [58]:
# Calcular TF-IDF: ingeniería de características de peliculas
tfidf_vectorizer_peliculas = TfidfVectorizer()

tfidf_matrix_peliculas = tfidf_vectorizer_peliculas.fit_transform(df_filtered['Lemmatized_Description'])

In [ ]:
# Define el modelo y el tokenizador
modelo = 'Helsinki-NLP/opus-mt-es-en'
tokenizer = MarianTokenizer.from_pretrained(modelo)
model = MarianMTModel.from_pretrained(modelo)

In [38]:
!pip install sentence-transformers

from sentence_transformers import SentenceTransformer, util
modelo = SentenceTransformer('msmarco-MiniLM-L-6-v3')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.72k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [39]:
# Generar incrustaciones para todas las consultas y respuestas
incrustaciones_consultas = modelo.encode("I'd like to watch a horror film")
incrustaciones_respuestas = modelo.encode(df['Description'])

In [ ]:
#pasarle todo el data, transformar la columna summary a un description

# Encontrar la respuesta con la mejor similitud para cada consulta
#for i, incrustacion_consulta in enumerate(incrustaciones_consultas):
similitudes = util.cos_sim(incrustaciones_consultas, incrustaciones_respuestas)[0]
#mejor_indice = similitudes.argmax()
#print(f"Consulta: '¿")
mejor_indice = similitudes.argmax().item()  # Convertir a entero
print(f"Mejor respuesta (Similitud: {similitudes[mejor_indice]:.4f}): {df_peliculas['Description'].iloc[mejor_indice]}")

print()


Mejor respuesta (Similitud: 0.3947): A documentary team films the lives of a group of vampires for a few months. The vampires share a house in Wellington, New Zealand. Turns out vampires have their own domestic problems too.



- juntar datasets.
- crear columna o estructura que le pase description a estado de animo para machear.
- que machee con el estado de ánimo en primer lugar, luego con el libro, juego o pelicula que le siga.
- acomodar los data en el archivo de datasets.
- informe.


# Modelado y funciones auxiliares

In [16]:
"""
Celda de traducción de frases
Se utiliza el modelo de traducción automática MarianMT
"""

# Define el modelo y el tokenizador
modelo = 'Helsinki-NLP/opus-mt-es-en'
tokenizer = MarianTokenizer.from_pretrained(modelo)
model = MarianMTModel.from_pretrained(modelo)

def translator(frase_usuario: str) -> str:
    """
    Traduce una frase de español a inglés.
    """
    # Tokeniza el texto y genera la traducción
    inputs = tokenizer(frase_usuario, return_tensors="pt")
    outputs = model.generate(**inputs)

    # Decodifica y muestra la traducción
    texto_ingles = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return texto_ingles

In [20]:
def sentiment_input() -> str:
  """
  Inicia el programa y pide al usuario que ingrese un prompt.
  """
  print('¡Bienvenido al recomendador de actividades!')
  sentiment_input = input("¿Cómo te sientes hoy?: ")

  return translator(sentiment_input)

In [69]:
def prompt_input() -> str:
  """
  Pide al usuario que ingrese su temática a explorar.
  """
  preference_input = input("¿Que temática te interesaría abordar?: ")

  return translator(preference_input)

In [ ]:
def procesar_frase_usuario(text: str) -> str:
    """
    Función para procesar una frase ingresada por el usuario.
    """
    text_stopwords = remove_stopwords(text)
    text_lemmatized = text_lemmatizer(text_stopwords)

    return text


## Análisis de sentimientos

In [9]:
# Cargamos el tokenizador y el modelo.
model_name: str = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer: BertTokenizer = BertTokenizer.from_pretrained(model_name)
model: BertForSequenceClassification = BertForSequenceClassification.from_pretrained(model_name)


# Creación de pipeline.
nlp: pipeline = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [23]:
def get_sentiment(text: str) -> str:
    """
    Retorna el sentimiento de un texto, mediante el uso de transformers.
    """
    label: str = nlp(text)[0]['label']
    score: float = nlp(text)[0]['score']

    if label in ['5 stars', '4 stars']:
        # print(score)
        return 'Alegre'
    elif label == '3 stars':
        # print(score)
        return 'Ni fu ni fa'
    elif label in ['2 stars', '1 star']:
        # print(score)
        return 'Melancólico'


### Prueba del modelo de analisis de sentimientos

In [11]:
test_list: list[str] = [
    "Estoy muy feliz",
    "Me siento entusiasmado",
    "Estoy tranquilo",
    "Me siento relajado",
    "Estoy aburrido",
    "Me siento nostálgico",
    "Estoy un poco triste",
    "Me siento melancólico",
    "Estoy frustrado",
    "Me siento ansioso",
    "Estoy enojado",
    "Me siento decepcionado",
    "Estoy muy emocionado",
    "Me siento inspirado",
    "Estoy cansado",
    "Me siento agotado",
    "Estoy preocupado",
    "Me siento optimista",
    "Estoy motivado",
    "Me siento agradecido",
    "Estoy desconcentrado",
    "Me siento esperanzado",
    "Estoy estresado",
    "Me siento insatisfecho",
    "Estoy satisfecho",
    "Me siento apático",
    "Estoy en paz",
    "Me siento irritado",
    "Estoy un poco nervioso",
    "Me siento confiado",
    "Estoy renovado",
    "Me siento pleno",
    "Estoy resignado",
    "Me siento inquieto",
    "Estoy contento",
    "Me siento vulnerable",
    "Estoy enamorado",
    "Me siento frustrado",
    "Estoy ilusionado",
    "Me siento reflexivo",
    "Estoy agradecido",
    "Me siento emocionalmente agotado"
]

for text in test_list:
  print(f'{text} -> {get_sentiment(text)}')

0.7204190492630005
Estoy muy feliz -> Alegre
0.5104050636291504
Me siento entusiasmado -> Alegre
0.4654117822647095
Estoy tranquilo -> Alegre
0.3073946535587311
Me siento relajado -> Melancólico
0.6528457999229431
Estoy aburrido -> Melancólico
0.32323989272117615
Me siento nostálgico -> Alegre
0.5161958932876587
Estoy un poco triste -> Ni fu ni fa
0.3490581214427948
Me siento melancólico -> Melancólico
0.6465574502944946
Estoy frustrado -> Melancólico
0.45507076382637024
Me siento ansioso -> Alegre
0.6622191667556763
Estoy enojado -> Melancólico
0.469738245010376
Me siento decepcionado -> Melancólico
0.6982787847518921
Estoy muy emocionado -> Alegre
0.38369399309158325
Me siento inspirado -> Alegre
0.6665818095207214
Estoy cansado -> Melancólico
0.40258100628852844
Me siento agotado -> Melancólico
0.35649794340133667
Estoy preocupado -> Melancólico
0.30568787455558777
Me siento optimista -> Ni fu ni fa
0.4021136164665222
Estoy motivado -> Alegre
0.45900627970695496
Me siento agradecido


### Analisis de sentimiento del contenido a partir de la columna 'Category'

A partir de las categorías/generos que abarca el contenido, se clasificará para que lo recomiende dependiendo los sentimientos

In [71]:
df_labeled = df_filtered.copy()

In [72]:
df_labeled['Sentiment'] = df_filtered['Category'].apply(get_sentiment)

In [75]:
df_labeled['Sentiment'].value_counts()

,count
Sentiment,
Alegre,2340
Ni fu ni fa,550
Melancólico,48


In [74]:
df_labeled[['Title', 'Category', 'Sentiment']].sample(20)

,Title,Category,Sentiment
903,Arcadia Quest: Inferno,"['Adventure', 'Fantasy', 'Fighting', 'Miniatur...",Alegre
868,Cry Havoc,"['Fighting', 'Miniatures', 'Science Fiction', ...",Alegre
1563,Wreck-It Ralph,"['Animation', 'Adventure', 'Comedy']",Alegre
1892,Real Steel,"['Action', 'Drama', 'Family']",Alegre
2293,"The Rape of the Lock, and Other Poems by Alexa...",['English poetry -- 18th century'],Ni fu ni fa
1961,Into the Forest,"['Drama', 'Sci-Fi', 'Thriller']",Alegre
287,Lewis & Clark: The Expedition,"['American West', 'Exploration']",Alegre
94,Age of Innovation,"['Civilization', 'Fantasy', 'Territory Building']",Alegre
2345,The Ten Books on Architecture by Vitruvius Pollio,['Architecture -- Early works to 1800'],Alegre
236,Nations,"['Card Game', 'Civilization', 'Economic']",Alegre



#### Analisis de sentimientos (segundo enfoque)

A fin de realizar una comparación, utilizamos sentiment-analysis sobre la misma lista de test.

In [35]:
sentiment = sentiment_analysis.SentimentAnalysisSpanish()
for texto in test_list:
  print(f'{texto} -> {sentiment.sentiment(texto)}')

Estoy muy feliz -> 0.6889487711193526
Me siento entusiasmado -> 0.0728752984896165
Estoy tranquilo -> 0.7826378949098491
Me siento relajado -> 0.23951579470116582
Estoy aburrido -> 0.31062133515766494
Me siento nostálgico -> 0.0728752984896165
Estoy un poco triste -> 0.04078651378014973
Me siento melancólico -> 0.0728752984896165
Estoy frustrado -> 0.31062133515766494
Me siento ansioso -> 0.0728752984896165
Estoy enojado -> 0.31062133515766494
Me siento decepcionado -> 0.005644903897282337
Estoy muy emocionado -> 0.537734459784737
Me siento inspirado -> 0.0728752984896165
Estoy cansado -> 0.31062133515766494
Me siento agotado -> 0.0728752984896165
Estoy preocupado -> 0.31062133515766494
Me siento optimista -> 0.0728752984896165
Estoy motivado -> 0.31062133515766494
Me siento agradecido -> 0.0728752984896165
Estoy desconcentrado -> 0.31062133515766494
Me siento esperanzado -> 0.0728752984896165
Estoy estresado -> 0.31062133515766494
Me siento insatisfecho -> 0.0728752984896165
Estoy sat

El rendimiento obtenido es inferior, teniendo en cuenta que un resultado cercano a uno es un texto positivo, cercano a cero es negativo y en los valores medios es neutro. Por eso se optó por los transformers.

Para obtener esta conclusión se visitó la [Documentación](https://pypi.org/project/sentiment-analysis-spanish/).


## Obtención de recomendaciones

In [ ]:
# Calcular TF-IDF
tfidf_vectorizer_peliculas = TfidfVectorizer()

tfidf_matrix_peliculas = tfidf_vectorizer_peliculas.fit_transform(df_labeled['Lemmatized_description'])

In [ ]:
def obtener_similitud(frase_usuario: str) -> pd.DataFrame:
    """
    Calcula la similitud de coseno entre una frase ingresada por el usuario
    y las descripciones de las películas usando la matriz TF-IDF.
    """
    # Procesar la frase del usuario
    frase_usuario = procesar_frase_usuario(frase_usuario)
    # Transformar la frase del usuario en la misma representación TF-IDF
    tfidf_usuario = tfidf_vectorizer_peliculas.transform([frase_usuario])

    # Calcular la similitud de coseno entre la frase del usuario y la matriz TF-IDF de las películas
    similitudes = cosine_similarity(tfidf_usuario, tfidf_matrix_peliculas)

    # Crear un DataFrame con los resultados, para ver las similitudes
    df_similitudes = pd.DataFrame(similitudes.T, columns=['similitud'], index=df_peliculas.index)

    # Agregar las películas a la tabla
    df_similitudes['titulo'] = df_peliculas['Title']

    # Ordenar por similitud de mayor a menor
    df_similitudes = df_similitudes.sort_values(by='similitud', ascending=False)

    return df_similitudes

In [ ]:
def get_prompt_recommendation(sentiment: str, preference: str) -> list[str]:
  """
  Recibe el sentimiento y el prompt de preferencia, devuelve una lista de strings con las recomendaciones.
  """
  pass


# Código programa

In [21]:
sentiment: str = sentiment_input()

¡Bienvenido al recomendador de actividades!
¿Cómo te sientes hoy?: Estoy muy contento


In [19]:
sentiment

'Happy.'

In [24]:
sentiment_labeled: str = get_sentiment(sentiment)
sentiment_labeled

'Alegre'

In [ ]:
# agregar analisis sentimientos